In [7]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import sys
sys.path.append('../..')
import pandas as pd
import seaborn as sns
import Model.input_pipeline
from Model.results import get_df, get_stats, final_plot

# Helper functions

In [8]:
def plus_encode_data(data):
    data = tf.reshape(data, shape=(data.shape[0], -1))
    mean_data = tf.reduce_mean(data, axis=1, keepdims=True)
    need_change = tf.cast(mean_data < 0.5, data.dtype)
    data = data * (1 - 2 * need_change) + need_change
    data = tf.reshape(data, shape=(-1, 32, 32, 1))
    return data   

def sample(num, generator, noise_dim=100):
    random_vector_for_generation = tf.random.normal([num, 1, 1, noise_dim])
    return generator(random_vector_for_generation)

def random_swap(data):
    data = tf.reshape(data, shape = (data.shape[0], -1))
    swap_mask = 2 * (np.random.random(size = (data.shape[0], 1)) > 0.5) - 1
    data = tf.multiply(data, swap_mask)
    data = tf.reshape(data, shape = (-1, 32, 32, 1))
    return data

def load_model_at_T(t, folder):
    generator = tf.keras.models.load_model(f"results/{folder}/{t:.1f}/generator")
    return generator

## Models dictionary

# Compare with monte carlo

# Data

In [ ]:
data_list = []
with tf.device('CPU:0'):
    for temp in temps:
        # Monte Carlo
        trainset_path = f"../../GetData/Python/Data/Data{temp:.2}.tfrecord"
        batch_size = 100
        train_set = Model.input_pipeline.dataset_tfrecord_pipeline(trainset_path, flatten=False, batch_size=batch_size)
        model = load_model_at_T(temp, folder)
        
        sample_list = []
        mc_list = []
        for batch in train_set:
            mc_list.append(batch)
            sample_list.append(sample(batch_size, model))
        
        data_mc = 2 * np.concatenate(mc_list, axis=0) - 1
        spin_sample =  2 * np.round(np.concatenate(sample_list, axis=0)) - 1
        spin_sample = random_swap(spin_sample)

        df_monte_carlo = get_df(data_mc)
        mc_stats = get_stats(df_monte_carlo, temp)
        mc_stats.extend((temp, "Monte Carlo"))
    
        df_sampled = get_df(spin_sample)
        sample_stats = get_stats(df_sampled, temp)
        sample_stats.extend((temp, "GAN"))
    
        data_list.append(mc_stats)
        data_list.append(sample_stats)

In [ ]:
final_plot(data_list)
plt.savefig("final_plot.svg")

## Examples

In [ ]:
fig, axes = plt.subplots(nrows=10, ncols=4, figsize=(10, 20))
for i, ax_row in enumerate(axes):
    ax_row[0].set_ylabel(titles[i], rotation=90, size='large')
    predictions = sample(5, models[temps[i]])
    for j, ax in enumerate(ax_row):
        ax.yaxis.set_ticklabels([])
        ax.xaxis.set_ticklabels([])
        spins = np.random.binomial(1, predictions[j, :, :])
        ax.imshow(spins, vmin=0, vmax=1)

plt.tight_layout()
plt.savefig("example.svg")